In [ ]:
import random
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [ ]:
random.seed(0)

In [ ]:
df_training = pd.read_csv('../../datasets/titanic_training_processed.csv')
df_test = pd.read_csv('../../datasets/titanic_test_processed.csv')

This is just an experiment to practice with the concept of statistical distances. The idea is to visualise a dimensionality reduced representation of the two classes (survived / non-survived) to get some awarenenss of their separability and then use a simple Mahalanobis distance between each test observation and the two classes to perform classification. 

Originally I wanted to apply t-SNE for dimensionality reduction, but the t-SNE algorithm in scikit-learn cannot be applied to new data. Therefore, I am sticking with PCA instead. 

In [ ]:
X = df_training.values[:, 2:]
pca = PCA(n_components = 2)
X_pca = pca.fit_transform(X)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(X_pca[df_training.Survived == 1, 0], X_pca[df_training.Survived == 1, 1], c = 'blue', alpha = 0.5)
ax.scatter(X_pca[df_training.Survived == 0, 0], X_pca[df_training.Survived == 0, 1], c = 'red', alpha = 0.5)

There seems to be a large overlap between the rwo distributions. What is the Mahalanobis distance value?

In [ ]:
discrete = np.arange(-3, 9, 0.1)
n_1 = np.sum(df_training.Survived == 1)
n_0 = np.sum(df_training.Survived == 0)
for i in discrete[1:]:
    for j in discrete[1:]:
        print() 

In [ ]:
np.sum(np.logical_and(X[:, 0] < 4, np.logical_and(X[:, 0] >= 3, np.logical_and(X[:, 1] < 2, X[:, 1] >= 1))))